In [55]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from keras.applications.resnet import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.applications import ResNet50, ResNet101
from keras.losses import CategoricalCrossentropy
import tensorflow as tf
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm, trange

In [26]:
# instantiate ResNet50 model
model50 = ResNet50(weights='imagenet')
model50.trainable = False

# instantiate ResNet101 model
model101 = ResNet101(weights='imagenet')
model101.trainable = False

In [48]:
IMG_PATH = 'bernie.png'
LABELS_PATH = 'labels.txt'
SIDE = 224
ALPHA = 0.5
IMAGES_BATCH = 200

with open(LABELS_PATH) as f:
    labels = f.read()
    labels = eval(labels)

In [70]:
resnet101_target = 742
resnet50_target = 353

In [50]:
base_img = image.load_img(IMG_PATH)
base_img = image.img_to_array(base_img)
assert base_img.shape == (SIDE, SIDE, 3)

In [51]:
def get_rnd_noise_img():
    return np.random.uniform(0, 255, (SIDE, SIDE, 3))

def mix_images(img1, img2, alpha):
    return alpha * img1 + (1 - alpha) * img2

def get_batch(img):
    return np.array([mix_images(img, get_rnd_noise_img(), ALPHA) for _ in range(IMAGES_BATCH)])

In [ ]:
def fool_brute_force(nsteps):
    count = defaultdict(int)
    for i in trange(nsteps):
        try:
            if i % 10 == 9:
                print(count)

            batch = get_batch(base_img)
            pred50 = model50.predict(batch, batch_size=IMAGES_BATCH, verbose=0)
            pred101 = model101.predict(batch, batch_size=IMAGES_BATCH, verbose=0)
            pred50 = [x[0][1] for x in decode_predictions(pred50, top=1)]
            pred101 = [x[0][1] for x in decode_predictions(pred101, top=1)]
            for p in pred50:
                count[p] += 1
            for p in pred101:
                count[p] += 1
            if resnet101_target in pred101:
                idx = pred101.index(resnet101_target)
                print(pred101[idx])
                image.save_img('found101.png', batch[idx])
            if resnet50_target in pred50:
                idx = pred50.index(resnet50_target)
                print(pred50[idx])
                image.save_img('found50.png', batch[idx])
            for idx, (p50, p100) in enumerate(zip(pred50, pred101)):
                if p50 == resnet50_target and p100 == resnet101_target:
                    print('Found image')
                    image.save_img('found.png', batch[idx])
                    print(batch[idx])
                    break
        except KeyboardInterrupt:
            break

In [56]:
loss_object = CategoricalCrossentropy()

def create_adversarial_pattern(model, input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = model.predict(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [78]:
label = tf.one_hot(resnet101_target, 1000)
label = tf.reshape(label, (1, 1000))

perturbation = create_adversarial_pattern(model101, preprocess_input(base_img)[None, ...], label)

ValueError: Passed in object [[[[-616.57306 -696.453   -750.76   ]
   [-616.57306 -696.453   -750.76   ]
   [-616.57306 -696.453   -750.76   ]
   ...
   [-565.573   -649.453   -702.76   ]
   [-565.573   -649.453   -702.76   ]
   [-565.573   -649.453   -702.76   ]]

  [[-615.573   -695.453   -749.76   ]
   [-615.573   -695.453   -749.76   ]
   [-615.573   -695.453   -749.76   ]
   ...
   [-565.573   -649.453   -702.76   ]
   [-565.573   -649.453   -702.76   ]
   [-565.573   -649.453   -702.76   ]]

  [[-614.573   -694.453   -748.76   ]
   [-614.573   -694.453   -748.76   ]
   [-614.573   -694.453   -748.76   ]
   ...
   [-565.573   -649.453   -702.76   ]
   [-565.573   -649.453   -702.76   ]
   [-565.573   -649.453   -702.76   ]]

  ...

  [[-562.573   -647.453   -696.76   ]
   [-568.573   -653.453   -702.76   ]
   [-556.573   -641.453   -690.76   ]
   ...
   [-703.57306 -790.453   -834.76   ]
   [-711.57306 -798.453   -842.76   ]
   [-719.57306 -806.453   -850.76   ]]

  [[-561.573   -